# Making files for other programs
This is an R Notebook detailing how I take a filtered .vcf file from ipyrad and turn it into other formats for population genetic analyses. I do this first for the full SNP dataset, then again for only outlier SNPs or only neutral SNPs so it is repetetive.

In [1]:
Sys.time()

[1] "2018-09-07 06:21:16 CDT"

## x45m75-neut

## Make .genind object for PCA and other conversions
I use the radiator package in R to convert the filtered .vcf file into a number of other input formats. It has great documentation and a lot of filtering capabilities itself.

In [3]:
library("adegenet") #inputting genetic data from structure, PCA, DAPC
library("radiator") #Conversion from vcf to a lot of other formats
library("dplyr")

Loading required package: ade4

   /// adegenet 2.1.1 is loaded ////////////

   > overview: '?adegenet'
   > tutorials/doc/questions: 'adegenetWeb()' 
   > bug reports/feature requests: adegenetIssues()



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
#set filename suffix
filtsuf = "OL-c85t10-x45m75-maf025-neutI2"

In [4]:
rad.filt <- vcf2genind("OL-c85-t10-x45m75-maf025-neutI2.recode.vcf",filename=filtsuf,
                      common.markers = T, strata="../Making_Files/OL-c85-t10-x45.strata", 
                      pop.levels = c("Klaskino_BC","Barkeley_BC","Ladysmith_BC","Victoria_BC","Discovery_WA","Liberty_WA","TritonCove_WA","NorthBay_WA","Willapa_WA","Netarts_OR","Yaquina_OR","Coos_OR","Humboldt_CA","Tomales_CA","NorthSanFran_CA","SouthSanFran_CA","Elkhorn_CA","MuguLagoon_CA","SanDiego_CA"),
                      parallel.core = 1)

VCF is biallelic


Reading 13232 / 13232 loci.
Done.


    number of markers with REF/ALT change(s) = 0
Using markers common in all populations:
    Number of markers before/blacklisted/after:13232/41/13191
Scanning for monomorphic markers...
    Number of markers before/blacklisted/after: 13191/0/13191

Tidy genomic data:
    Number of common markers: 13191
    Number of chromosome/contig/scaffold: 6095
    Number of individuals: 117
    Number of populations: 19

Writing tidy data set:
OL-c85t10-x45m75-maf025-neutI2.rad


In [4]:
rad.u <- vcf2genind(paste0(filtsuf,"-u.vcf"),filename=paste0(filtsuf,"-u"),
                      common.markers = T, strata="../Making_Files/OL-c85-t10-x45.strata", 
                      pop.levels = c("Klaskino_BC","Barkeley_BC","Ladysmith_BC","Victoria_BC","Discovery_WA","Liberty_WA","TritonCove_WA","NorthBay_WA","Willapa_WA","Netarts_OR","Yaquina_OR","Coos_OR","Humboldt_CA","Tomales_CA","NorthSanFran_CA","SouthSanFran_CA","Elkhorn_CA","MuguLagoon_CA","SanDiego_CA"),
                      parallel.core = 1)

VCF is biallelic


Reading 6110 / 6110 loci.
Done.


    number of markers with REF/ALT change(s) = 0
Using markers common in all populations:
    Number of markers before/blacklisted/after:6110/16/6094
Scanning for monomorphic markers...
    Number of markers before/blacklisted/after: 6094/0/6094

Tidy genomic data:
    Number of common markers: 6094
    Number of chromosome/contig/scaffold: 6094
    Number of individuals: 117
    Number of populations: 19

Writing tidy data set:
OL-c85t10-x45m75-maf025-neutI2-u.rad


Read in the .strata file and extract information such as region, latitude, and longitude. Region in my case was determined after running Structure analyses and exploring the data with PCA. For clarity, I show how to create the input files when region is already determined.

In [6]:
info <- as.data.frame(read.table("../Making_Files/OL-c85-t10-x45.strata",header = T,sep = "\t",stringsAsFactors = F))

mystrats <- as.data.frame(matrix(,nrow = length(indNames(rad.u$genind.no.imputation)),ncol=5))
colnames(mystrats) <- c("POPULATION","LOCATION","REGION","LATITUDE","LONGITUDE")

for(i in 1:nrow(info)){
    j <- grep(gsub("_","-",info[i,1]),indNames(rad.u$genind.no.imputation),value=FALSE)
    mystrats[j,1] <-info$STRATA[i] 
    mystrats[j,2] <-info$LOCATION[i] 
    mystrats[j,3] <-info$REGION[i]
    mystrats[j,4] <-info$LATITUDE[i]
    mystrats[j,5] <-info$LONGITUDE[i]
}
just.strats <- select(mystrats,c("POPULATION","LOCATION","REGION"))
stratted.u <- strata(rad.u$genind.no.imputation, formula= REGION/POPULATION/LOCATION, combine = TRUE,just.strats)
stratted.u@other <- select(mystrats, LATITUDE,LONGITUDE)


In [6]:
save(stratted.u, file=paste(filtsuf,"-u.genind",sep=""))

In [7]:
stratted.filt <- strata(rad.filt$genind.no.imputation, formula= REGION/POPULATION/LOCATION, combine = TRUE,just.strats)
stratted.filt@other <- select(mystrats, LATITUDE,LONGITUDE)
save(stratted.filt, file=paste(filtsuf,"-filt.genind",sep=""))

## Make hierfstat 
(because hierfstat will not accept this genind format). Make it for unlinked and all loci.

In [10]:
hf.u <- vcf2hierfstat(paste0(filtsuf,"-u.vcf"),filename=paste0(filtsuf,"-u"),
                      common.markers = T, strata="../Making_Files/OL-c85-t10-x45.strata", 
                      pop.levels = c("Klaskino_BC","Barkeley_BC","Ladysmith_BC","Victoria_BC","Discovery_WA","Liberty_WA","TritonCove_WA","NorthBay_WA","Willapa_WA","Netarts_OR","Yaquina_OR","Coos_OR","Humboldt_CA","Tomales_CA","NorthSanFran_CA","SouthSanFran_CA","Elkhorn_CA","MuguLagoon_CA","SanDiego_CA"),
                      parallel.core = 1)

VCF is biallelic


Reading 6110 / 6110 loci.
Done.


    number of markers with REF/ALT change(s) = 0
Using markers common in all populations:
    Number of markers before/blacklisted/after:6110/16/6094
Scanning for monomorphic markers...
    Number of markers before/blacklisted/after: 6094/0/6094

Tidy genomic data:
    Number of common markers: 6094
    Number of chromosome/contig/scaffold: 6094
    Number of individuals: 117
    Number of populations: 19
    * Number of sample pop, np = 19
    * Number of markers, nl = 6094
    * The highest number used to label an allele, nu = 4
    * The alleles are encoded with one digit number

Writing tidy data set:
OL-c85t10-x45m75-maf025-neutI2-u.rad


In [11]:
hf.u <- hf.u$hierfstat.no.imputation
save(hf.u, file=paste0(filtsuf,"-u.hf"))

In [12]:
hf.filt <- vcf2hierfstat("OL-c85-t10-x45m75-maf025-neutI2.recode.vcf",filename=paste0(filtsuf,"-filt"),
                      common.markers = T, strata="../Making_Files/OL-c85-t10-x45.strata", 
                      pop.levels = c("Klaskino_BC","Barkeley_BC","Ladysmith_BC","Victoria_BC","Discovery_WA","Liberty_WA","TritonCove_WA","NorthBay_WA","Willapa_WA","Netarts_OR","Yaquina_OR","Coos_OR","Humboldt_CA","Tomales_CA","NorthSanFran_CA","SouthSanFran_CA","Elkhorn_CA","MuguLagoon_CA","SanDiego_CA"),
                      parallel.core = 1)

VCF is biallelic


Reading 13232 / 13232 loci.
Done.


    number of markers with REF/ALT change(s) = 0
Using markers common in all populations:
    Number of markers before/blacklisted/after:13232/41/13191
Scanning for monomorphic markers...
    Number of markers before/blacklisted/after: 13191/0/13191

Tidy genomic data:
    Number of common markers: 13191
    Number of chromosome/contig/scaffold: 6095
    Number of individuals: 117
    Number of populations: 19
    * Number of sample pop, np = 19
    * Number of markers, nl = 13191
    * The highest number used to label an allele, nu = 4
    * The alleles are encoded with one digit number

Writing tidy data set:
OL-c85t10-x45m75-maf025-neutI2-filt.rad


In [13]:
hf.filt <- hf.filt$hierfstat.no.imputation

In [14]:
save(hf.filt, file=paste0(filtsuf,"-filt.hf"))

### Make input for Structure

In [15]:
#Use the previously created tidy datasets from radiator
write_structure(rad.filt$tidy.data, filename=paste0(filtsuf,"-filt"))

### Make input for Treemix
Use only the unlinked dataset. Uses a python script I wrote to convert a genpop object to Treemix input.

In [5]:
gp <- genind2genpop(stratted.u,pop=strata(stratted.u)$POPULATION)
write.table(gp$tab, file=paste0(filtsuf,"-u.gp"),sep = "\t",row.names = T,col.names = T,quote = F )
system('python ../../Methods/Scripts/genpop2Treemix.py OL-c85t10-x45m75-maf025-neutI2-u.gp OL-c85t10-x45m75-maf025-neutI2-u.pop.TM')
system('gzip OL-c85t10-x45m75-maf025-neutI2-u.pop.TM')


 Converting data from a genind to a genpop object... 

...done.



## Make EEMS input file 

Code to calculate the .diffs matrix for EEMS, from EEMS github.

In [8]:
bed2diffs_v1 <- function(Geno) {
  nIndiv <- nrow(Geno)
  nSites <- ncol(Geno)
  Diffs <- matrix(0, nIndiv, nIndiv)
  
  for (i in seq(nIndiv - 1)) {
    for (j in seq(i + 1, nIndiv)) {
      x <- Geno[i, ]
      y <- Geno[j, ]
      Diffs[i, j] <- mean((x - y)^2, na.rm = TRUE)
      Diffs[j, i] <- Diffs[i, j]
    }
  }
  Diffs
}
bed2diffs_v2 <- function(Geno) {
  nIndiv <- nrow(Geno)
  nSites <- ncol(Geno)
  Miss <- is.na(Geno)
  ## Impute NAs with the column means (= twice the allele frequencies)
  Mean <- matrix(colMeans(Geno, na.rm = TRUE), ## a row of means
                 nrow = nIndiv, ncol = nSites, byrow = TRUE) ## a matrix with nIndiv identical rows of means
  Mean[Miss == 0] <- 0 ## Set the means that correspond to observed genotypes to 0
  Geno[Miss == 1] <- 0 ## Set the missing genotypes to 0 (used to be NA) 
  Geno <- Geno + Mean
  ## Compute similarities
  Sim <- Geno %*% t(Geno) / nSites
  SelfSim <- diag(Sim) ## self-similarities
  vector1s <- rep(1, nIndiv) ## vector of 1s
  ## This chunk generates a `diffs` matrix
  Diffs <- SelfSim %*% t(vector1s) + vector1s %*% t(SelfSim) - 2 * Sim
  Diffs
}

Excluding Coos Bay, as we now know it is an anthropogenic migration event.
### Maf 0.025 - All SNPs

In [9]:
gind.xOR <- stratted.filt[!(indNames(stratted.filt) %in% c("OR1-1-C6","OR1-2-C5","OR1-6-C3","OR1-7-C6","OR1-12-C8","OR1-4-C5")),drop=TRUE]

suf <- "OLxOR1-t10x45m75-maf025-neutI2-filt"
geno <- gind.xOR@tab
stopifnot(identical(gind.xOR@type, 'codom'))
# Get rid of non-biallelic loci
multi.loci <- names(which(gind.xOR@loc.n.all != 2))
multi.cols <- which(grepl(paste0("^", multi.loci, "\\.\\d+$", collapse = "|"), colnames(geno)))
if (length(multi.cols)) geno <- geno[, - multi.cols]
nloci <- dim(geno)[2] / 2
#Choose allele to be "derived" allele.
geno <- geno[, c(seq(1,ncol(geno),by = 2))]

dim(geno)

[1]   111 13191

In [10]:
# 111 inds, 13,191 loci
#bed2diffs functions  
diffs.v1 <- bed2diffs_v1(geno)
diffs.v2 <- bed2diffs_v2(geno)
diffs.v1 <- round(diffs.v1, digits = 6)
diffs.v2 <- round(diffs.v2, digits = 6)

Check that the dissimilarity matrix has one positive eigenvalue and nIndiv-1 negative eigenvalues, as required by a full-rank Euclidean distance matrix.

In [11]:
tail(sort(round(eigen(diffs.v1)$values, digits = 2)))

[1] -0.22 -0.20 -0.17 -0.15 -0.01 57.00

In [21]:
tail(sort(round(eigen(diffs.v2)$values, digits = 2)))

[1] -0.18 -0.17 -0.17 -0.17 -0.04 48.56

Write files for V1.

In [12]:
write.table(diffs.v1, paste(suf,".v1.diffs",sep=""), 
            col.names = FALSE, row.names = FALSE, quote = FALSE)

In [13]:
## Get gps coordinates.
xOR.info <- dplyr::filter(info, !grepl("OR1",INDIVIDUALS))
gps_matrix <- matrix(,nrow = length(indNames(gind.xOR)),ncol=2)

for(i in 1:nrow(xOR.info)){
    j <- grep(gsub("_","-",xOR.info[i,1]),indNames(gind.xOR),value=FALSE)
    gps_matrix[j,1] <-xOR.info$LONGITUDE[i]
    gps_matrix[j,2] <-xOR.info$LATITUDE[i]
}

In [14]:
#write .coord file
write.table(gps_matrix, paste(suf,".v1.coord",sep=""),col.names = FALSE, row.names = FALSE,quote = FALSE)

# Outlier

Read in the .strata file and extract information such as region, latitude, and longitude. Region in my case was determined after running Structure analyses and exploring the data with PCA. For clarity, I show how to create the input files when region is already determined.

In [20]:
#set filename suffix
filtsuf = "OL-c85t10-x45m75-maf025-outI2Union"

In [21]:
rad.filt <- vcf2genind("OL-c85-t10-x45m75-maf025-outI2Union.recode.vcf",filename=filtsuf,
                      common.markers = T, strata="../Making_Files/OL-c85-t10-x45.strata", 
                      pop.levels = c("Klaskino_BC","Barkeley_BC","Ladysmith_BC","Victoria_BC","Discovery_WA","Liberty_WA","TritonCove_WA","NorthBay_WA","Willapa_WA","Netarts_OR","Yaquina_OR","Coos_OR","Humboldt_CA","Tomales_CA","NorthSanFran_CA","SouthSanFran_CA","Elkhorn_CA","MuguLagoon_CA","SanDiego_CA"),
                      parallel.core = 1)

VCF is biallelic


Reading 168 / 168 loci.
Done.


    number of markers with REF/ALT change(s) = 0
Using markers common in all populations:
    Number of markers before/blacklisted/after:168/0/168
Scanning for monomorphic markers...
    Number of markers before/blacklisted/after: 168/0/168

Tidy genomic data:
    Number of common markers: 168
    Number of chromosome/contig/scaffold: 97
    Number of individuals: 117
    Number of populations: 19

Writing tidy data set:
OL-c85t10-x45m75-maf025-outI2Union.rad


In [22]:
rad.u <- vcf2genind("OL-c85t10-x45m75-maf025-outI2Union-u.vcf",filename=paste0(filtsuf,"-u"),
                      common.markers = T, strata="../Making_Files/OL-c85-t10-x45.strata", 
                      pop.levels = c("Klaskino_BC","Barkeley_BC","Ladysmith_BC","Victoria_BC","Discovery_WA","Liberty_WA","TritonCove_WA","NorthBay_WA","Willapa_WA","Netarts_OR","Yaquina_OR","Coos_OR","Humboldt_CA","Tomales_CA","NorthSanFran_CA","SouthSanFran_CA","Elkhorn_CA","MuguLagoon_CA","SanDiego_CA"),
                      parallel.core = 1)

VCF is biallelic


Reading 97 / 97 loci.
Done.


    number of markers with REF/ALT change(s) = 0
Using markers common in all populations:
    Number of markers before/blacklisted/after:97/0/97
Scanning for monomorphic markers...
    Number of markers before/blacklisted/after: 97/0/97

Tidy genomic data:
    Number of common markers: 97
    Number of chromosome/contig/scaffold: 97
    Number of individuals: 117
    Number of populations: 19

Writing tidy data set:
OL-c85t10-x45m75-maf025-outI2Union-u.rad


Read in the .strata file and extract information such as region, latitude, and longitude. Region in my case was determined after running Structure analyses and exploring the data with PCA. For clarity, I show how to create the input files when region is already determined.

In [23]:
info <- as.data.frame(read.table("../Making_Files/OL-c85-t10-x45.strata",header = T,sep = "\t",stringsAsFactors = F))

mystrats <- as.data.frame(matrix(,nrow = length(indNames(rad.filt$genind.no.imputation)),ncol=5))
colnames(mystrats) <- c("POPULATION","LOCATION","REGION","LATITUDE","LONGITUDE")

for(i in 1:nrow(info)){
    j <- grep(gsub("_","-",info[i,1]),indNames(rad.filt$genind.no.imputation),value=FALSE)
    mystrats[j,1] <-info$STRATA[i] 
    mystrats[j,2] <-info$LOCATION[i] 
    mystrats[j,3] <-info$REGION[i]
    mystrats[j,4] <-info$LATITUDE[i]
    mystrats[j,5] <-info$LONGITUDE[i]
}
just.strats <- select(mystrats,c("POPULATION","LOCATION","REGION"))
stratted.filt <- strata(rad.filt$genind.no.imputation, formula= REGION/POPULATION/LOCATION, combine = TRUE,just.strats)
stratted.filt@other <- select(mystrats, LATITUDE,LONGITUDE)


In [24]:
save(stratted.filt, file=paste(filtsuf,"-filt.genind",sep=""))

In [25]:
stratted.u <- strata(rad.u$genind.no.imputation, formula= REGION/POPULATION/LOCATION, combine = TRUE,just.strats)
stratted.u@other <- select(mystrats, LATITUDE,LONGITUDE)
save(stratted.u, file=paste(filtsuf,"-u.genind",sep=""))

## Make hierfstat 
(because hierfstat will not accept this genind format). Make it for unlinked and all loci.

In [8]:
hf.filt <- vcf2hierfstat("OL-c85-t10-x45m75-maf025-outI2Union.recode.vcf",filename=paste0(filtsuf,"-filt"),
                      common.markers = T, strata="../Making_Files/OL-c85-t10-x45.strata", 
                      pop.levels = c("Klaskino_BC","Barkeley_BC","Ladysmith_BC","Victoria_BC","Discovery_WA","Liberty_WA","TritonCove_WA","NorthBay_WA","Willapa_WA","Netarts_OR","Yaquina_OR","Coos_OR","Humboldt_CA","Tomales_CA","NorthSanFran_CA","SouthSanFran_CA","Elkhorn_CA","MuguLagoon_CA","SanDiego_CA"),
                      parallel.core = 1)

VCF is biallelic


Reading 168 / 168 loci.
Done.


    number of markers with REF/ALT change(s) = 0
Using markers common in all populations:
    Number of markers before/blacklisted/after:168/0/168
Scanning for monomorphic markers...
    Number of markers before/blacklisted/after: 168/0/168

Tidy genomic data:
    Number of common markers: 168
    Number of chromosome/contig/scaffold: 97
    Number of individuals: 117
    Number of populations: 19
    * Number of sample pop, np = 19
    * Number of markers, nl = 168
    * The highest number used to label an allele, nu = 4
    * The alleles are encoded with one digit number

Writing tidy data set:
OL-c85t10-x45m75-maf025-outI2Union-filt.rad


In [9]:
hf.filt <- hf.filt$hierfstat.no.imputation

In [10]:
save(hf.filt, file=paste0(filtsuf,"-filt.hf"))

### Make input for Structure

In [11]:
#Use the previously created tidy datasets from radiator
write_structure(rad.filt$tidy.data, filename=paste0(filtsuf,"-filt"))

### Make input for Treemix
Use only the unlinked dataset. Uses a python script I wrote to convert a genpop object to Treemix input.

In [12]:
gp <- genind2genpop(stratted.filt,pop=strata(stratted.filt)$POPULATION)
write.table(gp$tab, file=paste0(filtsuf,"-u.gp"),sep = "\t",row.names = T,col.names = T,quote = F )
system('python ../../Methods/Scripts/genpop2Treemix.py OL-c85t10-x45m75-maf025-outI2-u.gp OL-c85t10-x45m75-maf025-outI2-u.pop.TM')
system('gzip OL-c85t10-x45m75-maf025-outI2-u.pop.TM')


 Converting data from a genind to a genpop object... 

...done.



## Make EEMS input file 

Code to calculate the .diffs matrix for EEMS, from EEMS github.

In [17]:
bed2diffs_v1 <- function(Geno) {
  nIndiv <- nrow(Geno)
  nSites <- ncol(Geno)
  Diffs <- matrix(0, nIndiv, nIndiv)
  
  for (i in seq(nIndiv - 1)) {
    for (j in seq(i + 1, nIndiv)) {
      x <- Geno[i, ]
      y <- Geno[j, ]
      Diffs[i, j] <- mean((x - y)^2, na.rm = TRUE)
      Diffs[j, i] <- Diffs[i, j]
    }
  }
  Diffs
}
bed2diffs_v2 <- function(Geno) {
  nIndiv <- nrow(Geno)
  nSites <- ncol(Geno)
  Miss <- is.na(Geno)
  ## Impute NAs with the column means (= twice the allele frequencies)
  Mean <- matrix(colMeans(Geno, na.rm = TRUE), ## a row of means
                 nrow = nIndiv, ncol = nSites, byrow = TRUE) ## a matrix with nIndiv identical rows of means
  Mean[Miss == 0] <- 0 ## Set the means that correspond to observed genotypes to 0
  Geno[Miss == 1] <- 0 ## Set the missing genotypes to 0 (used to be NA) 
  Geno <- Geno + Mean
  ## Compute similarities
  Sim <- Geno %*% t(Geno) / nSites
  SelfSim <- diag(Sim) ## self-similarities
  vector1s <- rep(1, nIndiv) ## vector of 1s
  ## This chunk generates a `diffs` matrix
  Diffs <- SelfSim %*% t(vector1s) + vector1s %*% t(SelfSim) - 2 * Sim
  Diffs
}

Excluding Coos Bay, as we now know it is an anthropogenic migration event.
### Maf 0.025 - All SNPs

In [18]:
gind.xOR <- stratted.filt[!(indNames(stratted.filt) %in% c("OR1-1-C6","OR1-2-C5","OR1-6-C3","OR1-7-C6","OR1-12-C8","OR1-4-C5")),drop=TRUE]

suf <- "OL-xOR1-t10x45m75-maf025-neutI2-filt"
geno <- gind.xOR@tab
stopifnot(identical(gind.xOR@type, 'codom'))
# Get rid of non-biallelic loci
multi.loci <- names(which(gind.xOR@loc.n.all != 2))
multi.cols <- which(grepl(paste0("^", multi.loci, "\\.\\d+$", collapse = "|"), colnames(geno)))
if (length(multi.cols)) geno <- geno[, - multi.cols]
nloci <- dim(geno)[2] / 2
#Choose allele to be "derived" allele.
geno <- geno[, c(seq(1,ncol(geno),by = 2))]

dim(geno)

[1]   111 13191

In [19]:
# 111 inds, 13,191 loci
#bed2diffs functions  
diffs.v1 <- bed2diffs_v1(geno)
diffs.v2 <- bed2diffs_v2(geno)
diffs.v1 <- round(diffs.v1, digits = 6)
diffs.v2 <- round(diffs.v2, digits = 6)

Check that the dissimilarity matrix has one positive eigenvalue and nIndiv-1 negative eigenvalues, as required by a full-rank Euclidean distance matrix.

In [20]:
tail(sort(round(eigen(diffs.v1)$values, digits = 2)))

[1] -0.22 -0.20 -0.17 -0.15 -0.01 57.00

In [21]:
tail(sort(round(eigen(diffs.v2)$values, digits = 2)))

[1] -0.18 -0.17 -0.17 -0.17 -0.04 48.56

Write files for V1.

In [22]:
write.table(diffs.v1, paste(suf,".v1.diffs",sep=""), 
            col.names = FALSE, row.names = FALSE, quote = FALSE)

In [23]:
## Get gps coordinates.
xOR.info <- dplyr::filter(info, !grepl("OR1",INDIVIDUALS))
gps_matrix <- matrix(,nrow = length(indNames(gind.xOR)),ncol=2)

for(i in 1:nrow(xOR.info)){
    j <- grep(gsub("_","-",xOR.info[i,1]),indNames(gind.xOR),value=FALSE)
    gps_matrix[j,1] <-xOR.info$LONGITUDE[i]
    gps_matrix[j,2] <-xOR.info$LATITUDE[i]
}

In [24]:
#write .coord file
write.table(gps_matrix, paste(suf,".v1.coord",sep=""),col.names = FALSE, row.names = FALSE,quote = FALSE)

## Plot EEMS Results
Do after you have run EEMS.

In [1]:
library(rEEMSplots) #Get from EEMS github
library(rgdal)
library(rworldmap)
library(rworldxtra)

Loading required package: sp
rgdal: version: 1.2-18, (SVN revision 718)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 1.11.3, released 2015/09/16
 Path to GDAL shared files: /usr/share/gdal/1.11
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.2, 08 September 2015, [PJ_VERSION: 492]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.2-7 
### Welcome to rworldmap ###
For a short introduction type : 	 vignette('rworldmap')


#### Maf 0.025

Make a list of all EEMS directories to plot all the results into 1 plot:

In [5]:
path = "../EEMS/OLxOR1-x45m75-maf025-filt_results/"
suf = "OLxOR1-x45m75-maf025-filt-SkI"
dirs = c(paste0(path,suf,"-nD200-ch1"),paste0(path,suf,"-nD200-ch2"),paste0(path,suf,"-nD200-ch3"),
         paste0(path,suf,"-nD250-ch1"),paste0(path,suf,"-nD250-ch2"),paste0(path,suf,"-nD250-ch3"),
         paste0(path,suf,"-nD300-ch1"),paste0(path,suf,"-nD300-ch2"),paste0(path,suf,"-nD300-ch3"),
         paste0(path,suf,"-nD350-ch1"),paste0(path,suf,"-nD350-ch2"),paste0(path,suf,"-nD350-ch3"),
         paste0(path,suf,"-nD400-ch1"),paste0(path,suf,"-nD400-ch2"),paste0(path,suf,"-nD400-ch3"),
         paste0(path,suf,"-nD500-ch1"),paste0(path,suf,"-nD500-ch2"),paste0(path,suf,"-nD500-ch3"),
        paste0(path,suf,"-nD600-ch1"),paste0(path,suf,"-nD600-ch2"),paste0(path,suf,"-nD600-ch3"),
         paste0(path,suf,"-nD700-ch1"),paste0(path,suf,"-nD700-ch2"),paste0(path,suf,"-nD700-ch3"))

In [7]:
eems.plots(mcmcpath = dirs, plotpath = paste0(path,"OLxOR1-x45m75-maf025-SkI-All-plots-noD"),
           longlat = T,add.grid=F,add.outline = T,add.demes = F,
           projection.in = "+proj=longlat +datum=WGS84",projection.out = "+proj=merc +datum=WGS84",
           add.map = T,add.abline = T, add.r.squared = T)

Input projection: +proj=longlat +datum=WGS84
Output projection: +proj=merc +datum=WGS84



Loading rgdal (required by projection.in)
Loading rworldmap (required by add.map)
Loading rworldxtra (required by add.map)
Using the default DarkOrange to Blue color scheme, with 'white' as the midpoint color.
It combines two color schemes from the 'dichromat' package, which itself is based on
a collection of color schemes for scientific data graphics:
	Light A and Bartlein PJ (2004). The End of the Rainbow? Color Schemes for Improved Data
	Graphics. EOS Transactions of the American Geophysical Union, 85(40), 385.
See also http://geog.uoregon.edu/datagraphics/color_scales.htm



Using 'euclidean' distance to assign interpolation points to Voronoi tiles.



Processing the following EEMS output directories :
../EEMS/OLxOR1-x45m75-maf025-filt_results/OLxOR1-x45m75-maf025-filt-SkI-nD200-ch1../EEMS/OLxOR1-x45m75-maf025-filt_results/OLxOR1-x45m75-maf025-filt-SkI-nD200-ch2../EEMS/OLxOR1-x45m75-maf025-fi

In [13]:
for(dir in dirs){
    eems.plots(mcmcpath = dir, plotpath = paste0(dir,"-plots"),
           longlat = T,add.grid=F,add.outline = T,add.demes = T,
           projection.in = "+proj=longlat +datum=WGS84",projection.out = "+proj=merc +datum=WGS84",
           add.map = T,add.abline = T, add.r.squared = T)
}


Input projection: +proj=longlat +datum=WGS84
Output projection: +proj=merc +datum=WGS84



Loading rgdal (required by projection.in)
Loading rworldmap (required by add.map)
Loading rworldxtra (required by add.map)
Using the default DarkOrange to Blue color scheme, with 'white' as the midpoint color.
It combines two color schemes from the 'dichromat' package, which itself is based on
a collection of color schemes for scientific data graphics:
	Light A and Bartlein PJ (2004). The End of the Rainbow? Color Schemes for Improved Data
	Graphics. EOS Transactions of the American Geophysical Union, 85(40), 385.
See also http://geog.uoregon.edu/datagraphics/color_scales.htm



Using 'euclidean' distance to assign interpolation points to Voronoi tiles.



Processing the following EEMS output directories :
../../EEMS/Choosing/OLxOR1-m80x60-maf025_results/OLxOR1-m80x60-maf025-W-nD200-ch1
Plotting effective migration surface (posterior mean of m rates)
../../EEMS/Choosing/OLxOR1-m80x60-maf025_resul